In [ ]:
%pip install accelerate -U -qqq
%pip install transformers[torch] -qqq

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import yaml
from distutils.dir_util import copy_tree

In [3]:
import torch
import yaml
from src.data.datamodule import DataManager

In [4]:
import json
import os
import numpy as np
import re

from tqdm import tqdm

from src.data.mt_dataset import MTDataset_HF
from src.data.tokenizers.unif_tokenizers import UNIFTokenizer

In [5]:
data_config = yaml.load(open("configs/data_config.yaml", 'r', encoding='utf-8'), Loader=yaml.Loader)
data_path = lambda x: data_config["path_repository"] + "data/" + data_config["data_language"] + str(x) + data_config["data_name_file"] + ".json"

In [8]:
def prepare_sql_input(kb_id_list: list[str], db2attr_dict: dict[str:list[str]]) -> list[int]:
    input_list = []
    for kb_id in kb_id_list:
        final_input_str = ' [schema] '
        question_relevant_db_attributes = db2attr_dict[kb_id]
        final_input_str = final_input_str + " ".join(question_relevant_db_attributes)
        input_list.append(final_input_str)
    return input_list

In [13]:
config = data_config
device = "cpu"
db2attr_dict = json.load(open("data/table_id2new_attrs_for_parsing.json", 'r', encoding="utf8"))

tokenizer = UNIFTokenizer(path_tok=config["path_repository"] + "data/query_vocab.json",
                               pre_train_name=config["pre_train_tokenizer"],
                               pad_flag=True,
                               max_length=config["max_sent_len"])

def prepare_data(path_data, drop_last=False):

    dev_data = json.load(open(os.path.join(path_data), 'r', encoding="utf-8"))
    target_sentences = []
    source_sentences = []
    kb_id_sentences = []
    for sample in tqdm(dev_data[:config["separate_batch"]], desc="Pars data"):
        target_sentences.append(sample['masked_query'])
        source_sentences.append(sample['question'])
        kb_id_sentences.append(sample['kb_id'])
        
    kb_id_sentences = prepare_sql_input(kb_id_sentences, db2attr_dict)
    source_sentences = [i+j for i, j in zip(source_sentences, kb_id_sentences)]

    # DataLoader

    tokenized_source_sentences = [tokenizer.tkr(i) for i in source_sentences]
    tokenized_target_sentences = [tokenizer.tkr(i) for i in target_sentences]

    dataset = MTDataset_HF(tokenized_source_list=tokenized_source_sentences,
                        tokenized_target_list=tokenized_target_sentences, device=device)
    return dataset

In [14]:
dev_dataloader = prepare_data(path_data=data_path("train"), drop_last=False)
test_dataloader = prepare_data(path_data=data_path("dev"), drop_last=True) # dev

Pars data: 100%|██████████| 8420/8420 [00:00<00:00, 1213317.75it/s]


In [15]:
from transformers import XLMRobertaForCausalLM, AutoConfig
config = AutoConfig.from_pretrained("FacebookAI/roberta-base")
config.is_decoder = True
model = XLMRobertaForCausalLM.from_pretrained("FacebookAI/roberta-base", config=config)#.to('cuda')

In [16]:
sum(p.numel() for p in model.parameters())

124697433

In [17]:
from transformers import Trainer, TrainingArguments

In [18]:
training_args = TrainingArguments(
    f"roberta-base-exp",
    evaluation_strategy = "epoch",
    learning_rate=2e-5, # 2e-5,
    weight_decay=0.01,
    num_train_epochs = 20,
    logging_dir = 'logs',
    save_strategy="no" # "epoch", "no"
)

In [19]:
from transformers import ProgressCallback, PrinterCallback

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dev_dataloader,
    eval_dataset=test_dataloader,
    callbacks = [PrinterCallback]
)

In [21]:
end_train = trainer.train()

  0%|          | 500/140900 [01:08<5:12:25,  7.49it/s]

{'loss': 0.2617, 'learning_rate': 1.992902767920511e-05, 'epoch': 0.07}
{'loss': 0.2617, 'learning_rate': 1.992902767920511e-05, 'epoch': 0.07}


  1%|          | 1000/140900 [02:15<5:12:06,  7.47it/s]

{'loss': 0.1908, 'learning_rate': 1.9858055358410224e-05, 'epoch': 0.14}
{'loss': 0.1908, 'learning_rate': 1.9858055358410224e-05, 'epoch': 0.14}


  1%|          | 1501/140900 [03:22<5:15:43,  7.36it/s]

...

{'loss': 0.0813, 'learning_rate': 1.27750177430802e-07, 'epoch': 19.87}
{'loss': 0.0813, 'learning_rate': 1.27750177430802e-07, 'epoch': 19.87}


100%|█████████▉| 140501/140900 [5:26:06<00:53,  7.42it/s]

{'loss': 0.0797, 'learning_rate': 5.6777856635912e-08, 'epoch': 19.94}
{'loss': 0.0797, 'learning_rate': 5.6777856635912e-08, 'epoch': 19.94}


100%|█████████▉| 1049/1053 [00:40<00:00, 25.47it/s]
                                                         
100%|██████████| 140900/140900 [5:27:40<00:00,  7.17it/s]

{'eval_loss': 0.19474487006664276, 'eval_runtime': 41.0893, 'eval_samples_per_second': 204.92, 'eval_steps_per_second': 25.627, 'epoch': 20.0}
{'eval_loss': 0.19474487006664276, 'eval_runtime': 41.0893, 'eval_samples_per_second': 204.92, 'eval_steps_per_second': 25.627, 'epoch': 20.0}
{'train_runtime': 19660.9621, 'train_samples_per_second': 57.326, 'train_steps_per_second': 7.166, 'train_loss': 0.10940194611180497, 'epoch': 20.0}
{'train_runtime': 19660.9621, 'train_samples_per_second': 57.326, 'train_steps_per_second': 7.166, 'train_loss': 0.10940194611180497, 'epoch': 20.0}


In [22]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

100%|██████████| 1053/1053 [00:41<00:00, 25.66it/s]

{'eval_loss': 0.19474487006664276, 'eval_runtime': 41.0559, 'eval_samples_per_second': 205.086, 'eval_steps_per_second': 25.648, 'epoch': 20.0}
Perplexity: 1.22


In [23]:
import random

In [24]:
len(dev_dataloader)

56354

In [25]:
trainer.model.to('cpu');
for _ in range(5):
    index = random.randint(0,len(test_dataloader)-1)
    print(f"---------true-----------")
    print(tokenizer.decode(test_dataloader[index]['input_ids']), tokenizer.decode(test_dataloader[index]['labels']))
    print(f"---------predict-----------")
    predict = trainer.model.generate(test_dataloader[index]['input_ids'].reshape((1,-1)), max_new_tokens=40, do_sample=True, top_k=50, top_p=0.95)[0]
    print(tokenizer.decode(predict))

---------true-----------


IndexError: list index out of range

In [ ]:
trainer.model.to('cpu');
for _ in range(5):
    index = random.randint(0,len(test_dataloader)-1)
    print(f"---------true-----------")
    print(tokenizer.decode(test_dataloader[index]['input_ids']), tokenizer.decode(test_dataloader[index]['labels']))
    print(f"---------predict-----------")
    predict = trainer.model.generate(test_dataloader[index]['input_ids'].reshape((1,-1)), max_new_tokens=40, do_sample=True, top_k=50, top_p=0.95)[0]
    print(tokenizer.decode(predict))

In [ ]:
trainer.model.to('cpu');
for _ in range(4):
    index = random.randint(0,len(dev_dataloader)-1)
    print(f"---------true-----------")
    print(tokenizer.decode(dev_dataloader[index]['input_ids']), tokenizer.decode(dev_dataloader[index]['labels']))
    print(f"---------predict-----------")
    predict = trainer.model.generate(dev_dataloader[index]['input_ids'].reshape((1,-1)), max_new_tokens=40, do_sample=True, top_k=50, top_p=0.95)[0]
    print(tokenizer.decode(predict))

In [ ]:
metrics = end_train.metrics

# save train results
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

In [24]:
test_model = XLMRobertaForCausalLM.from_pretrained("/home/jupyter/datasphere/project/roberta-base-exp/checkpoint-42270")#.to('cuda')

You are using a model of type roberta to instantiate a model of type xlm-roberta. This is not supported for all configurations of models and can yield errors.


In [25]:
config = data_config
device = "cpu"

tokenizer = UNIFTokenizer(path_tok=config["path_repository"] + "data/query_vocab.json",
                               pre_train_name=config["pre_train_tokenizer"],
                               pad_flag=False,
                               max_length=config["max_sent_len"])

def prepare_data(path_data, drop_last=False):

    dev_data = json.load(open(os.path.join(path_data), 'r', encoding="utf-8"))
    target_sentences = []
    source_sentences = []
    for sample in tqdm(dev_data[:config["separate_batch"]], desc="Pars data"):
        target_sentences.append(sample['masked_query'])
        source_sentences.append(sample['question'])

    # DataLoader

    tokenized_source_sentences = [tokenizer.tkr(i) for i in source_sentences][0:40]
    tokenized_target_sentences = [tokenizer.tkr(i) for i in target_sentences][0:40]

    dataset = MTDataset_HF(tokenized_source_list=tokenized_source_sentences,
                        tokenized_target_list=tokenized_target_sentences, device=device)
    return dataset

In [26]:
test_model.to('cpu');
for _ in range(4):
    index = random.randint(0,len(dev_dataloader)-1)
    print(f"---------true-----------")
    print(tokenizer.decode(dev_dataloader[index]['input_ids']), tokenizer.decode(dev_dataloader[index]['labels']))
    print(f"---------predict-----------")
    predict = test_model.generate(dev_dataloader[index]['input_ids'].reshape((1,-1)), max_new_tokens=40, do_sample=True, top_k=50, top_p=0.95)[0]
    print(tokenizer.decode(predict))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


---------true-----------
<s>Which Attendance has an Opponent of carolina panthers ?</s> <s>SELECT Attendance FROM table WHERE Opponent = STR_VALUE_1</s>
---------predict-----------


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s>Which Attendance has an Opponent of carolina panthers ?</s> insertiononentonentonentinonentonent_inininonentonentonent_in
---------true-----------
<s>What is the number of played when points against 645 ?</s> <s>SELECT COUNT Played FROM table WHERE Pts_Agst = NUM_VALUE_1</s>
---------predict-----------


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s>What is the number of played when points against 645 ?</s></s>
---------true-----------
<s>What is tuesday day three when thursday day five is kamis ?</s> <s>SELECT Tuesday_Day_Three FROM table WHERE Thursday_Day_Five = STR_VALUE_1</s>
---------predict-----------


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s>What is tuesday day three when thursday day five is kamis ?</s>1VALUE1</s>
---------true-----------
<s>In what parish is the sub-parish fortun ? </s> <s>SELECT Parish_(Prestegjeld) FROM table WHERE Sub-Parish_(Sogn) = STR_VALUE_1</s>
---------predict-----------
<s>In what parish is the sub-parish fortun ? </s>1 WHERE = WHERE)Par_ParParParParParParParParParParParParParParParParParParParParPar_PPParParPar
